# Volatility Forecasting

In this project, you will perform a comparative analysis of various volatility forecasting methods:

1. close-to-close
2. parkinson
3. garman-klass
4. rogers-sachell
5. yang-zang
6. garch(1, 1)



**Part 1: Code Implementation and Verification (Jupyter Notebook) -- 100 pts**
1. Implement the all six estimators (close-to-close, parkinson, garman-klass, rogers-sachell, yang-zang and garch(1, 1)) in code.
2. For the backtest period starting from 1/1/2005 and ending on 4/2/2016, calculate the regression coefficient and $R^2$ metric for *weekly* forecasts for each of the estimators, and compare your results to Sepp's (pg. 41). Do this ONLY for SPY.

**Grading Rubric:**
- accuracy of results: 50 pts
- code commenting: 25 pts
- readability of notebook: 25 pts


# 1.

In [23]:
import numpy as np
import pandas as pd
from yfinance import download as yfinance_download
from pandas_datareader import data as datareader
from sklearn.linear_model import LinearRegression as LinReg
from arch import arch_model as garch_mod

# Setting options for better DataFrame visualization
pd.set_option('display.max_ro\ws', 10)

def close_to_close(returns_data):
    # Total observations
    total_observations = returns_data.shape[0]

    # Mean of returns
    mean_return = returns_data.mean()

    # Variance and annualization
    annualized_volatility = np.sqrt((1 / (total_observations - 1)) * ((returns_data - mean_return) ** 2).sum()) * np.sqrt(252)

    return annualized_volatility

def Parkinson(df_data):
    # Extracting high and low prices
    high_prices = df_data.high
    low_prices = df_data.low

    park_vol = (1 / (4 * np.log(2))) * np.sum((np.log(high_prices / low_prices)) ** 2) * np.sqrt(252)
    
    return park_vol


def Garman_Klass(df_data):
    # Extracting necessary price data
    open_prices = df_data.open
    close_prices = df_data.close
    high_prices = df_data.high
    low_prices = df_data.low

    gk_vol = np.sum(0.5 * (np.log(high_prices / low_prices)) ** 2 - ((2 * np.log(2) - 1) * (np.log(close_prices / open_prices)) ** 2)) * np.sqrt(252)

    return gk_vol

def Rogers_Satchell(df_data):
    # Extracting price data
    open_prices = df_data.open
    close_prices = df_data.close
    high_prices = df_data.high
    low_prices = df_data.low

    # Rogers-Satchell volatility refactor
    rs_vol = np.sum((np.log(high_prices / close_prices) * np.log(high_prices / open_prices)) + (np.log(low_prices / close_prices) * np.log(low_prices / open_prices))) * np.sqrt(252)

    return rs_vol


def Yang_Zhang(df_data):
    # Extract necessary data
    total_obs = df_data.shape[0]
    open_prices = df_data.open
    close_prices = df_data.close
    high_prices = df_data.high
    low_prices = df_data.low
    overnight_returns = df_data.overnight
    open_close_diff = df_data.open_close

    # Compute mean values
    avg_overnight = overnight_returns.mean()
    avg_open_close = open_close_diff.mean()

    # Constant factor
    constant_c = 0.34 / (1.34 + ((total_obs + 1) / (total_obs - 1)))

    # Rogers-Satchell part
    rs_part = np.sum((np.log(high_prices / close_prices) * np.log(high_prices / open_prices)) +
                            (np.log(low_prices / close_prices) * np.log(low_prices / open_prices)))

    # Variance components
    var_overnight = np.sum((overnight_returns - avg_overnight) ** 2)
    var_open_close = np.sum((open_close_diff - avg_open_close) ** 2)

    # Yang-Zhang final volatility
    yz_vol = (var_overnight + constant_c * var_open_close + (1 - constant_c) * rs_part) * np.sqrt(252)

    return yz_vol

# 2. 

In [63]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")

# Shared function for data fetching and preprocessing
def fetch_and_prepare_data(ticker, start='2004-12-31', end='2016-04-02'):
    df = yf.download(ticker, start=start, end=end).reset_index()
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df.rename(columns={'date': 'trade_date'}, inplace=True)
    df['dly_ret'] = np.log(df['close']).diff()
    df = df[1:].reset_index(drop=True)
    
    # Compute weekly number
    df['weekday'] = df['trade_date'].dt.weekday
    df['week_num'] = (df['weekday'].diff() < 0).cumsum()
    
    # Add week start and end dates
    df_start_end = (df.groupby('week_num', as_index=False)['trade_date']
                    .agg([min, max])
                    .rename(columns={'min': 'week_start', 'max': 'week_end'}))
    df = df.merge(df_start_end, left_on='week_num', right_index=True)
    
    return df

# Shared function for linear regression
def linear_regression(predicted_vol, realized_vol, name):
    X = predicted_vol.values.reshape(-1, 1)
    y = realized_vol.values.reshape(-1, 1)
    mdl_reg = LinearRegression().fit(X, y)
    
    return {
        f"R2_{name}": mdl_reg.score(X, y),
        f"Intercept_{name}": mdl_reg.intercept_[0],
        f"Slope_{name}": mdl_reg.coef_[0][0],
        f"Bias_{name}": np.mean(predicted_vol - realized_vol),
        f"Efficiency_{name}": np.std(realized_vol) / np.std(predicted_vol)
    }

# Shared function to compute realized volatility
def compute_realized_vol(df):
    return (df.groupby('week_num', as_index=False)['dly_ret']
            .agg(lambda x: np.std(x) * np.sqrt(252))
            .rename(columns={'dly_ret': 'realized_vol'}))[1:]

# Function for close-to-close estimator
def close_to_close_dataframe(ticker):
    df = fetch_and_prepare_data(ticker)
    df_realized = compute_realized_vol(df)
    
    # Predicted vol for close-to-close
    df_predicted = (df.groupby('week_num', as_index=False)['dly_ret']
                    .agg(lambda x: np.sqrt(np.sum(x**2)) * np.sqrt(252))
                    .rename(columns={'dly_ret': 'close_to_close'}))
    
    df_predicted = df_predicted[:-1]

    return linear_regression(df_predicted['close_to_close'], df_realized['realized_vol'], 'close_to_close')

# Parkinson estimator
def parkinson_dataframe(ticker):
    df = fetch_and_prepare_data(ticker)
    df_realized = compute_realized_vol(df)
    
    # Predicted vol for Parkinson
   # Group by 'week_num' and calculate Parkinson volatility in one step
    df_predicted = (df.groupby('week_num', as_index=False)
                  .apply(lambda x: pd.Series({'parkinson': np.sqrt((1/(4*np.log(2))) * np.mean((np.log(x['high'] / x['low'])) ** 2)) * np.sqrt(252)}))
                  .reset_index(drop=True))

    df_predicted = df_predicted[:-1]

    return linear_regression(df_predicted['parkinson'], df_realized['realized_vol'], 'parkinson')

# Garman-Klass estimator
def garman_dataframe(ticker):
    df = fetch_and_prepare_data(ticker)
    df_realized = compute_realized_vol(df)
    
    # Predicted vol for Garman-Klass
    df_predicted = (df.groupby('week_num', as_index=False)
                    .apply(lambda x: pd.Series({'garman':np.sqrt((0.5 * np.mean((np.log(x['high'] / x['low']))**2) 
                                              - (2*np.log(2) - 1) * np.mean((np.log(x['close'] / x['open']))**2))) * np.sqrt(252)}))
                    .reset_index(drop=True))
    df_predicted = df_predicted[:-1]

    return linear_regression(df_predicted['garman'], df_realized['realized_vol'], 'garman')

# Rogers-Satchell estimator
def rogers_dataframe(ticker):
    df = fetch_and_prepare_data(ticker)
    df_realized = compute_realized_vol(df)
    
    # Predicted vol for Rogers-Satchell
    df_predicted = (df.groupby('week_num', as_index=False)
                    .apply(lambda x: pd.Series({'rogers':np.sqrt(np.mean(np.log(x['high']/x['open']) * np.log(x['high']/x['close']) 
                                                       + np.log(x['low']/x['open']) * np.log(x['low']/x['close']))) * np.sqrt(252)}))
                    .reset_index(drop=True))
    df_predicted = df_predicted[:-1]

    return linear_regression(df_predicted['rogers'], df_realized['realized_vol'], 'rogers')

# Yang-Zhang estimator
def yan_zhang_dataframe(ticker):
    df = fetch_and_prepare_data(ticker)
    df_realized = compute_realized_vol(df)
    
    # Predicted vol for Yang-Zhang
    df_predicted = (df.groupby('week_num', as_index=False)
                    .apply(lambda x: pd.Series({'yan_zhang':np.sqrt(np.mean(np.log(x['open']/x['close'].shift(1))**2 
                                                     + np.mean(np.log(x['high']/x['low'])**2) 
                                                     + np.mean(np.log(x['close']/x['open'])**2))) * np.sqrt(252)}))
                    .reset_index())
    df_predicted = df_predicted[:-1]

    return linear_regression(df_predicted['yan_zhang'], df_realized['realized_vol'], 'yan_zhang')


In [64]:
close_to_close_df = pd.DataFrame.from_records([close_to_close_dataframe('SPY')])
parkinson_df = pd.DataFrame.from_records([parkinson_dataframe('SPY')])
garman_df = pd.DataFrame.from_records([garman_dataframe("SPY")])
rogers_df = pd.DataFrame.from_records([rogers_dataframe("SPY")])
yan_zhang_df = pd.DataFrame.from_records([yan_zhang_dataframe("SPY")])
garch_df = pd.DataFrame.from_records([garch_dataframe("SPY")])
# Combine each of dataframe of each ticker into one dataframe
combined_df = pd.concat([close_to_close_df, parkinson_df, garman_df, rogers_df, yan_zhang_df, garch_df], axis=1)
df = combined_df
df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


R2_close_to_close  Intercept_close_to_close  Slope_close_to_close  \
0           0.506755                  0.035033              0.301117   

   Bias_close_to_close  Efficiency_close_to_close  R2_parkinson  \
0               0.2032                   0.422996      0.612194   

   Intercept_parkinson  Slope_parkinson  Bias_parkinson  Efficiency_parkinson  \
0             0.013786         0.931186       -0.004732              1.190124   

   ...  Intercept_yan_zhang  Slope_yan_zhang  Bias_yan_zhang  \
0  ...             0.013716         0.458526        0.132575   

   Efficiency_yan_zhang  Ticker  R2_garch  Intercept_garch  Slope_garch  \
0              0.589217     SPY  0.563334        -0.010277     0.616698   

   Bias_garch  Efficiency_garch  
0    0.102227          0.821655  

[1 rows x 31 columns]

In [65]:
def restructure_df(df):
    # Reset the index and rename the resulting column to "Ticker"
    df_reset = df.reset_index().rename(columns={df.index.name: "Ticker"})
    
    # Melt the dataframe to go from wide to long format
    df_long = df_reset.melt(id_vars=['Ticker'])
    
    # Split the 'variable' column into 'Metric' and 'Estimator' using expand=True
    df_long[['Metric', 'Estimator']] = df_long['variable'].str.split('_', n=1, expand=True)
    
    # Pivot the table to have 'Estimator' as rows and 'Metric' as columns
    df_pivot = df_long.pivot_table(index='Estimator', columns='Metric', values='value', aggfunc='first')
    
    # Reorder columns for better visualization
    df_pivot = df_pivot[['R2', 'Intercept', 'Slope', 'Bias', 'Efficiency']]
    
    # Rename the index for better readability
    df_pivot = df_pivot.rename(index={
        'yan_zhang': 'Yan-Zhang',
        'close_to_close': 'Close-to-Close',
        'garch': 'Garch',
        'parkinson': 'Parkinson',
        'garman': 'Garman-Klass',
        'rogers': 'Rogers-Satchell'
    })
    
    return df_pivot

restructure_df(df)

Metric                 R2  Intercept     Slope      Bias  Efficiency
Estimator                                                           
Close-to-Close   0.506755   0.035033  0.301117  0.203200    0.422996
Garch            0.563334  -0.010277  0.616698  0.102227    0.821655
Garman-Klass     0.613024   0.014787  0.910322 -0.002770    1.162669
Parkinson        0.612194   0.013786  0.931186 -0.004732    1.190124
Rogers-Satchell  0.594207   0.020335  0.861468 -0.001545    1.117559
Yan-Zhang        0.605589   0.013716  0.458526  0.132575    0.589217

The $R^2$ are close to the Sepp's metric as shown below 

In [66]:
# get the metric from sepp's result 
data = {
    'Estimator': ['Close-Close', 'OHLC + Close-Open', 'Parkinson', 'Garman-Klass', 'Rogers-Satchell'],
    'R^2': [0.51,0.58, 0.54, 0.55, 0.57],  # Convert percentages to proportions
    'Intercept': [0.078,0.07, 0.074, 0.074, 0.073],  # Convert percentages to proportions
    'Slope': [0.45, 0.45, 0.55, 0.54, 0.52]
}

df_sepp = pd.DataFrame(data)
df_sepp.set_index('Estimator', inplace=True)
df_sepp


R^2  Intercept  Slope
Estimator                                
Close-Close        0.51      0.078   0.45
OHLC + Close-Open  0.58      0.070   0.45
Parkinson          0.54      0.074   0.55
Garman-Klass       0.55      0.074   0.54
Rogers-Satchell    0.57      0.073   0.52